In [8]:
import torch
import pickle
import os
from tqdm import tqdm
sys.path.append('/home/ql2221/Projects/thermalizer/thermalizer/kolmogorov')
import performance
sys.path.append('/home/ql2221/Projects/thermalizer/thermalizer/models')
import misc
sys.path.append('/home/ql2221/Projects/thermalizer/thermalizer/systems')
import training_systems

In [11]:
#get the emulator
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
file_string = '/scratch/ql2221/thermalizer_data/wandb_data/wandb/run-20250526_223850-r12kgbg1/files/checkpoint_best.p'
with open(file_string, 'rb') as f:
    checkpoint = pickle.load(f)

# Extract the configuration (this might include hyperparameters, model architecture details, etc.)
config = checkpoint['config']

In [12]:
model=misc.load_model(file_string).to(device)

In [2]:
checkpoint_string = "/scratch/ql2221/thermalizer_data/wandb_data/wandb/run-20250526_223850-r12kgbg1/files/checkpoint_last.p"
trainer = training_systems.trainer_from_checkpoint(checkpoint_string)
model= trainer.model

Prep data


/ext3/miniforge3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Prep model
Prep optimizer


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ql2221 (qiliu2221) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [13]:
#load data
import pickle

# Load data from a .pkl file
with open("/scratch/cp3759/thermalizer_data/kolmogorov/reynolds10k/test40.p", "rb") as f:
    loaded_data = pickle.load(f)

sim_stack_loaded = loaded_data["data"]

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = sim_stack_loaded[0,0:10000,:,:]
x = x.unsqueeze(0).to(device)
print(x.shape)

torch.Size([1, 10000, 64, 64])


In [15]:
emu_rollout=performance.EmulatorRollout(x,model,residual=True,sigma=None,silence=False)
emu_rollout.evolve()

100%|██████████| 9999/9999 [01:03<00:00, 157.18it/s]


In [16]:
def make_movie(state_vector, save_path="emulator_movie.mp4", fps=30, vmin=None, vmax=None, stride=10):
    """
    Create a movie from a 4D tensor of shape (batch, time, height, width), using only every `stride`-th frame.
    
    Parameters:
    - state_vector: torch.Tensor of shape (B, T, H, W)
    - save_path: output video file path
    - fps: frames per second for the movie
    - vmin, vmax: color scale
    - stride: only include every `stride`th frame in the video
    """
    B, T, H, W = state_vector.shape
    data = state_vector[0].cpu().numpy()

    fig, ax = plt.subplots()
    im = ax.imshow(data[0], cmap="viridis", vmin=vmin, vmax=vmax)
    ax.set_title("Timestep 0")
    fig.colorbar(im, ax=ax)

    # Frame indices to use
    frame_indices = list(range(0, T, stride))

    def update(frame_idx):
        frame = frame_indices[frame_idx]
        im.set_data(data[frame])
        ax.set_title(f"Timestep {frame}")
        return im,

    ani = animation.FuncAnimation(fig, update, frames=len(frame_indices), blit=False, repeat=False)
    ani.save(save_path, fps=fps)
    plt.close(fig)
    print("movie saved")

In [17]:
rollout = emu_rollout.emu
make_movie(rollout, save_path="rollout.mp4", fps=30)

movie saved


In [22]:
x = sim_stack_loaded[0,0:100,:,:]
x = x.to(device)
print(x.shape)

torch.Size([100, 64, 64])


In [23]:
roll = performance.KolmogorovAnimation(x,model,savestring="hh")

In [24]:
roll.animate()

saving
.

/home/ql2221/Projects/thermalizer/thermalizer/kolmogorov/performance.py:430: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(self.pred).float().unsqueeze(0).unsqueeze(0)
/home/ql2221/Projects/thermalizer/thermalizer/kolmogorov/performance.py:442: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


...........

IndexError: index 100 is out of bounds for dimension 0 with size 100